# Web Scraping using Python

## Import Libraries

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [3]:
# Request the webpage and get the text
urlreq = requests.get('https://en.wikipedia.org/wiki/Comparison_of_North_American_ski_resorts')

# Extract the text from the requested webpage
url = urlreq.text

# Create a parse tree with BS using the html parser
soup = BeautifulSoup(url, 'html.parser')

In [4]:
# Transforming html table into a dataframe
dfs = pd.read_html(url)
df = dfs[4] # 5th line
df

,Resort name and website,Nearest city,State/province,Peak elevation (ft),Base elevation (ft),Vertical drop (ft),Skiable acreage,Total trails,Total lifts,Avg annual snowfall (in),Adult weekend lift ticket window price (USD),Date statistics updated
0,Ski Bromont,Bromont,Quebec,1854,590.0,1264.0,450,141.0,9.0,190,$54,"December 1, 2019[1]"
1,Apex Mountain Resort,Penticton,British Columbia,7197,5197.0,2000.0,1112,79.0,4.0,236,$65,"November 30, 2019[2]"
2,Canyon Ski Area,Red Deer,Alberta,2950,2412.0,538.0,80,23.0,6.0,45,$35,"November 30, 2019[3]"
3,Fernie Alpine Resort,Fernie,British Columbia,7000,3450.0,3550.0,2500,142.0,10.0,360,$90,"November 30, 2019[4]"
4,Marble Mountain Ski Resort,Steady Brook,Newfoundland and Labrador,1791,33.0,1759.0,230,40.0,5.0,192,$34,"December 1, 2019[5]"
...,...,...,...,...,...,...,...,...,...,...,...,...
459,Sasquatch Mountain,Chehalis,British Columbia,NaN,NaN,1100.0,300,NaN,NaN,NaN,NaN,NaN
460,Mount Washington Alpine Resort,Courtenay,British Columbia,NaN,NaN,1663.0,1700,NaN,NaN,432,NaN,NaN
461,Manning Park,Manning Park,British Columbia,NaN,NaN,1417.0,140,NaN,NaN,296,NaN,NaN
462,Mount Baldy Ski Area,Oliver,British Columbia,NaN,NaN,1293.0,600,NaN,NaN,256,NaN,NaN


In [16]:
df.head()

,Resort,City,State,Peak elevation (ft),Base elevation (ft),Vertical drop (ft),Skiable acreage,Total trails,Total lifts,Avg annual snowfall (in),Lift ticket (USD)
0,Ski Bromont,Bromont,Quebec,1854,590,1264,450,141,9,190,$54
1,Apex Mountain Resort,Penticton,British Columbia,7197,5197,2000,1112,79,4,236,$65
2,Canyon Ski Area,Red Deer,Alberta,2950,2412,538,80,23,6,45,$35
3,Fernie Alpine Resort,Fernie,British Columbia,7000,3450,3550,2500,142,10,360,$90
4,Marble Mountain Ski Resort,Steady Brook,Newfoundland and Labrador,1791,33,1759,230,40,5,192,$34


## Data Cleaning and Manipulation

Web scraping was used to collect information from Wikipedia which was then transformed into a dataframe. However, this dataset is not available in the most useful format as is and must be cleaned before any analysis can be performed.  


In [5]:
# Rename columns
df.rename(columns = {'Resort name and website':'Resort', 'Nearest city':'City', 'State/province':'State', 'Adult weekend lift ticket window price (USD)':'Lift ticket (USD)'}, inplace=True)
# Drop columns that are not necessary 
df = df.drop(['Date statistics updated'], axis=1)

In [15]:
# Check for duplicates
df.duplicated().sum()

0

In [14]:
# Check for missing values
# df.isna().sum()
df.isna().sum().sort_values(ascending=False)

Avg annual snowfall (in)    67
Skiable acreage             31
Peak elevation (ft)         21
Lift ticket (USD)           10
Total lifts                  0
Total trails                 0
Vertical drop (ft)           0
Base elevation (ft)          0
State                        0
City                         0
Resort                       0
dtype: int64

In [13]:
# Check datatypes of columns
df.dtypes

Resort                      object
City                        object
State                       object
Peak elevation (ft)         object
Base elevation (ft)          int64
Vertical drop (ft)           int64
Skiable acreage             object
Total trails                 int64
Total lifts                  int64
Avg annual snowfall (in)    object
Lift ticket (USD)           object
dtype: object

### Missing Values

In [19]:
# Exclude rows with missing values from 'City' column 
df = df[df['City'].notna()] # only take values that are not missing

In [11]:
# Replace missing values in base elevation, total trails, total lifts, and vertical drop columns with mean of their respective column
df['Base elevation (ft)'].fillna(value=df['Base elevation (ft)'].mean(), inplace=True)
df['Vertical drop (ft)'].fillna(value=df['Vertical drop (ft)'].mean(), inplace=True)
df['Total trails'].fillna(value=df['Total trails'].mean(), inplace=True)
df['Total lifts'].fillna(value=df['Total lifts'].mean(), inplace=True)

In [12]:
# Change datatypes to 'int' to remove decimals
convert_dict = {'Base elevation (ft)': int, 'Total trails': int, 'Total lifts': int, 'Vertical drop (ft)': int}
# Instead of individual coloumns we use a dictionary to convert and reduce work
df = df.astype(convert_dict)

In [8]:
df

,Resort,City,State,Peak elevation (ft),Base elevation (ft),Vertical drop (ft),Skiable acreage,Total trails,Total lifts,Avg annual snowfall (in),Lift ticket (USD)
0,Ski Bromont,Bromont,Quebec,1854,590.0,1264.0,450,141.0,9.0,190,$54
1,Apex Mountain Resort,Penticton,British Columbia,7197,5197.0,2000.0,1112,79.0,4.0,236,$65
2,Canyon Ski Area,Red Deer,Alberta,2950,2412.0,538.0,80,23.0,6.0,45,$35
3,Fernie Alpine Resort,Fernie,British Columbia,7000,3450.0,3550.0,2500,142.0,10.0,360,$90
4,Marble Mountain Ski Resort,Steady Brook,Newfoundland and Labrador,1791,33.0,1759.0,230,40.0,5.0,192,$34
...,...,...,...,...,...,...,...,...,...,...,...
459,Sasquatch Mountain,Chehalis,British Columbia,NaN,NaN,1100.0,300,NaN,NaN,NaN,NaN
460,Mount Washington Alpine Resort,Courtenay,British Columbia,NaN,NaN,1663.0,1700,NaN,NaN,432,NaN
461,Manning Park,Manning Park,British Columbia,NaN,NaN,1417.0,140,NaN,NaN,296,NaN
462,Mount Baldy Ski Area,Oliver,British Columbia,NaN,NaN,1293.0,600,NaN,NaN,256,NaN


### Special Characters

In [32]:
# Removing special characters from 'Lift ticket' column and replacing missing values with mean
df['Lift ticket (USD)'] = df['Lift ticket (USD)'].replace('$', '', regex=True)
df['Lift ticket (USD)'] = df['Lift ticket (USD)'].fillna(-1)
df['Lift ticket (USD)'] = df['Lift ticket (USD)'].astype(str)
replacers = {'-1':float('nan'), 'Private Club':float('nan'), 'Free':float('nan'), 'Temporarily Closed':float('nan'), '25/season':float('nan'), '84.95[70]':float('84.95'), 'Closed Temporarily':float('nan')}
df['Lift ticket (USD)'] = df['Lift ticket (USD)'].replace(replacers)
df['Lift ticket (USD)'] = df['Lift ticket (USD)'].astype(float)
df['Lift ticket (USD)'].fillna(value=df['Lift ticket (USD)'].mean(), inplace=True)


In [34]:
# Change datatype to 'int' to remove decimals
df['Lift ticket (USD)'] = df['Lift ticket (USD)'].astype(int)

In [35]:
df

,Resort,City,State,Peak elevation (ft),Base elevation (ft),Vertical drop (ft),Skiable acreage,Total trails,Total lifts,Avg annual snowfall (in),Lift ticket (USD)
0,Ski Bromont,Bromont,Quebec,1854,590,1264,450,141,9,190.00000,54
1,Apex Mountain Resort,Penticton,British Columbia,7197,5197,2000,1112,79,4,236.00000,65
2,Canyon Ski Area,Red Deer,Alberta,2950,2412,538,80,23,6,45.00000,35
3,Fernie Alpine Resort,Fernie,British Columbia,7000,3450,3550,2500,142,10,360.00000,90
4,Marble Mountain Ski Resort,Steady Brook,Newfoundland and Labrador,1791,33,1759,230,40,5,192.00000,34
...,...,...,...,...,...,...,...,...,...,...,...
459,Sasquatch Mountain,Chehalis,British Columbia,NaN,3277,1100,300,43,7,186.41519,63
460,Mount Washington Alpine Resort,Courtenay,British Columbia,NaN,3277,1663,1700,43,7,432.00000,63
461,Manning Park,Manning Park,British Columbia,NaN,3277,1417,140,43,7,296.00000,63
462,Mount Baldy Ski Area,Oliver,British Columbia,NaN,3277,1293,600,43,7,256.00000,63


In [31]:
# Removing special characters from 'Avg annual snowfall' column and replacing missing values with mean
replacers1 = {'220[21]':float('220'), '80[58]':float('80'), '132[63]':float('132'), '180[69]':float('180'), '100[87]':float('100'), '120[89]':float('120'), '168[100]':float('168'), '8,464[102]':float('8464'), '311`':float('311'), '9,422[102]':float('9422')}
df['Avg annual snowfall (in)'] = df['Avg annual snowfall (in)'].replace(replacers1)
df['Avg annual snowfall (in)'] = df['Avg annual snowfall (in)'].astype(float)
df['Avg annual snowfall (in)'].fillna(value=df['Avg annual snowfall (in)'].mean(), inplace=True)


In [36]:
# Change datatype to 'int' to remove decimals
df['Avg annual snowfall (in)'] = df['Avg annual snowfall (in)'].astype(int)

In [37]:
# Removing special characters from 'Skiable acreage' column and replacing missing values with mean
df['Skiable acreage'] = df['Skiable acreage'].replace(replacers1)
df['Skiable acreage'] = df['Skiable acreage'].astype(float)
df['Skiable acreage'].fillna(value=df['Skiable acreage'].mean(), inplace=True)


In [38]:
df

,Resort,City,State,Peak elevation (ft),Base elevation (ft),Vertical drop (ft),Skiable acreage,Total trails,Total lifts,Avg annual snowfall (in),Lift ticket (USD)
0,Ski Bromont,Bromont,Quebec,1854,590,1264,450.0,141,9,190,54
1,Apex Mountain Resort,Penticton,British Columbia,7197,5197,2000,1112.0,79,4,236,65
2,Canyon Ski Area,Red Deer,Alberta,2950,2412,538,80.0,23,6,45,35
3,Fernie Alpine Resort,Fernie,British Columbia,7000,3450,3550,2500.0,142,10,360,90
4,Marble Mountain Ski Resort,Steady Brook,Newfoundland and Labrador,1791,33,1759,230.0,40,5,192,34
...,...,...,...,...,...,...,...,...,...,...,...
459,Sasquatch Mountain,Chehalis,British Columbia,NaN,3277,1100,300.0,43,7,186,63
460,Mount Washington Alpine Resort,Courtenay,British Columbia,NaN,3277,1663,1700.0,43,7,432,63
461,Manning Park,Manning Park,British Columbia,NaN,3277,1417,140.0,43,7,296,63
462,Mount Baldy Ski Area,Oliver,British Columbia,NaN,3277,1293,600.0,43,7,256,63


In [39]:
# Change datatype to 'int' to remove decimals
df['Skiable acreage'] = df['Skiable acreage'].astype(int)

In [40]:
# Removing special characters from 'Peak elevation' column and replacing missing values with mean
df['Peak elevation (ft)'] = df['Peak elevation (ft)'].replace(replacers1)
df['Peak elevation (ft)'] = df['Peak elevation (ft)'].astype(float)
df['Peak elevation (ft)'].fillna(value=df['Peak elevation (ft)'].mean(), inplace=True)

In [41]:
# Change datatype to 'int' to remove decimals 
df['Peak elevation (ft)'] = df['Peak elevation (ft)'].astype(int)                                         

In [42]:
# Confirming that all missing values are replaced
df.isna().sum().sort_values(ascending=False)

Lift ticket (USD)           0
Avg annual snowfall (in)    0
Total lifts                 0
Total trails                0
Skiable acreage             0
Vertical drop (ft)          0
Base elevation (ft)         0
Peak elevation (ft)         0
State                       0
City                        0
Resort                      0
dtype: int64

# Creating the csv file

In [ ]:
# Exporting data to a CSV 
df.to_csv('ski_resort_analysis.csv', index=False)